In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 경로
file_path = 'data/transfer_data_real_part_1.csv'
df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip')

FileNotFoundError: [Errno 2] No such file or directory: 'data/exports/split_files/transfer_data_real_part_1.csv'

In [ ]:
df

,type,txhash,timestamp,amount,denom,dpDenom,fromAddress,toAddress,fromChain,fromChainId,toChain,toChainId
0,Send,04A7F145D3530040025E99EF9203864015565E69E616AC...,1613651332000,0.000001,uatom,ATOM,cosmos1tssqr446rmp692ltagnjh7xk36gdaq5w7y0eqf,cosmos1gzc54u4p67hh9r4m9vcml3ke9fc29tplsefy6k,cosmos,cosmoshub-4,cosmos,cosmoshub-4
1,Send,9D4CAAB3CB1EB9685A3C56708BB97051519BA101C0D6A2...,1613651732000,1.000000,uatom,ATOM,cosmos1vv6hruquzpty4xpks9znkw8gys5x4nsnqw9f4k,cosmos1vv6hruquzpty4xpks9znkw8gys5x4nsnqw9f4k,cosmos,cosmoshub-4,cosmos,cosmoshub-4
2,Send,4804E4604A663474998148D48E740901D65C4E7FA2003D...,1613651752000,1.000000,uatom,ATOM,cosmos1vv6hruquzpty4xpks9znkw8gys5x4nsnqw9f4k,cosmos1vv6hruquzpty4xpks9znkw8gys5x4nsnqw9f4k,cosmos,cosmoshub-4,cosmos,cosmoshub-4
3,Send,7AA093F8DEDBF7CAE222B2CDFBEA23C858CC91496A36DB...,1613652734000,11.414900,uatom,ATOM,cosmos14pfpm03g35fduhukvhd390rw4hm7hh4jtrttny,cosmos1nm0rrq86ucezaf8uj35pq9fpwr5r82cl8sc7p5,cosmos,cosmoshub-4,cosmos,cosmoshub-4
4,Send,906DD734EEEA8A18F07879885438D1AAD87C56DD737F66...,1613652734000,39.110730,uatom,ATOM,cosmos1hg7un3v82ldq450apnw0dnhxp9948jpqmrhxry,cosmos1nm0rrq86ucezaf8uj35pq9fpwr5r82cl8sc7p5,cosmos,cosmoshub-4,cosmos,cosmoshub-4
...,...,...,...,...,...,...,...,...,...,...,...,...
11249771,IBCReceive,55CB09F8A9087CFB71EE773990D2B75677F8B72D25EAFF...,1644257151000,149533.500000,uhuahua,Unknown,chihuahua12sczd7vmheqat355txqducgm6fk86ye4jhp0xe,osmo12sczd7vmheqat355txqducgm6fk86ye4eel33f,chihuahua,chihuahua-1,osmosis,osmosis-1
11249772,IBCReceive,A35014DEC76C9141B64B0168D2EBFEC0A159C73B981EC5...,1644257151000,76.228050,uluna,Unknown,terra1p5lhpw87kdgfkhmclrv87kh68ckce5hptwuy92,osmo1p5lhpw87kdgfkhmclrv87kh68ckce5hp93453c,terra,phoenix-1,osmosis,osmosis-1
11249773,IBCReceive,D690A11CB522EA6AEFFA8223408EBC6725D64AEE661962...,1644257151000,149533.500000,uhuahua,Unknown,chihuahua12sczd7vmheqat355txqducgm6fk86ye4jhp0xe,osmo12sczd7vmheqat355txqducgm6fk86ye4eel33f,chihuahua,chihuahua-1,osmosis,osmosis-1
11249774,IBCReceive,F4DE12697FCF61412764059D98782FF34B529A472520BE...,1644257151000,76.228050,uluna,Unknown,terra1p5lhpw87kdgfkhmclrv87kh68ckce5hptwuy92,osmo1p5lhpw87kdgfkhmclrv87kh68ckce5hp93453c,terra,phoenix-1,osmosis,osmosis-1


In [ ]:
# IBCReceive 제거
df = df[df['type'] != 'IBCReceive']

In [ ]:
# 외부체인 거래인가?
df['is_external'] = df['fromChain'] != df['toChain']

/tmp/ipykernel_675883/4245052492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_external'] = df['fromChain'] != df['toChain']


In [ ]:
node_df = df

In [ ]:
external_df = node_df[node_df['is_external']]

In [ ]:
# 1. 보낸 트랜젝션 집계
sent_stats = (
    node_df
    .groupby('fromAddress')
    .agg(sent_tx_count = ('amount','count'),
         sent_tx_amount = ('amount','sum'))
    .rename_axis('address')
)

# 2. 받은 트랜젝션 집계
recv_stats = (
    node_df
    .groupby('toAddress')
    .agg(recv_tx_count = ('amount','count'),
         recv_tx_amount = ('amount','sum'))
    .rename_axis('address')
)

# 3. external sent 거래 집계
external_sent_stats = (
    external_df
    .groupby('fromAddress')
    .agg(external_sent_tx_count = ('amount','count'),
         external_sent_tx_amount = ('amount','sum'))
    .rename_axis('address')
)
# 4. external recv 거래 집계
external_recv_stats = (
    external_df
    .groupby('toAddress')
    .agg(external_recv_tx_count = ('amount','count'),
         external_recv_tx_amount = ('amount','sum'))
    .rename_axis('address')
)

# 1~4 종합
total_stats = (
    sent_stats
    .join(recv_stats, how='outer')
    .join(external_sent_stats, how='outer')
    .join(external_recv_stats, how='outer')
)

In [ ]:
total_stats.describe()

,sent_tx_count,sent_tx_amount,recv_tx_count,recv_tx_amount,external_sent_tx_count,external_sent_tx_amount,external_recv_tx_count,external_recv_tx_amount
count,8.410280e+05,8.410280e+05,1.540818e+06,1.540818e+06,234125.000000,2.341250e+05,262593.000000,2.625930e+05
mean,1.064476e+01,1.809675e+07,5.810251e+00,9.877787e+06,9.134227,2.435164e+05,8.143976,2.171165e+05
std,1.829368e+03,1.219137e+10,6.549561e+02,9.007032e+09,62.363414,6.208751e+07,53.466695,5.861841e+07
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00
25%,1.000000e+00,5.512641e+00,1.000000e+00,5.250000e+00,1.000000,1.000000e+01,1.000000,8.000000e+00
50%,1.000000e+00,4.259812e+01,1.000000e+00,5.254582e+01,3.000000,9.471753e+01,3.000000,6.947996e+01
75%,3.000000e+00,5.059988e+02,3.000000e+00,7.140216e+02,7.000000,1.399982e+03,6.000000,9.700000e+02
max,1.448407e+06,1.000000e+13,4.387820e+05,1.000000e+13,12220.000000,3.000000e+10,7350.000000,3.000000e+10


In [ ]:
# 5. 시간대별 entrophy 계산
temp_df = node_df
# 5-1. timestamp → hour
temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp'], unit = 'ms')
temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
temp_df['hour'] = temp_df['timestamp'].dt.hour

# 5-2. 보낸 트랜잭션만 추출
sent = (
    temp_df[['fromAddress','hour']]
    .rename(columns={'fromAddress':'address'})
)

# 5-3. hour별 보낸 횟수 집계
hourly = (
    sent
    .groupby(['address','hour'])
    .size()
    .rename('hour_count')
    .reset_index()
)

# 5-4. Shannon entropy 계산
def shannon_entropy(counts):
    p = counts / counts.sum()
    return -(p * np.log2(p)).sum()

ent = (
    hourly
    .groupby('address')['hour_count']
    .apply(shannon_entropy)
    .rename('hour_entropy')
    .reset_index()
)
time_stats = ent

# 5-5. 기존 total_stats 와 병합 
total_stats = (
    total_stats.reset_index()
               .merge(time_stats, on='address', how='left')
               .fillna(0)
               .set_index('address')
)

/tmp/ipykernel_675883/3721333743.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp'], unit = 'ms')
/tmp/ipykernel_675883/3721333743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
/tmp/ipykernel_675883/3721333743.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [ ]:
#6. 활동 일수 기반 꾸준지표 계산
date_stats = (
    temp_df
    .groupby('fromAddress')['timestamp']
    .agg(
        first_date       = 'min',
        last_date        = 'max',
        active_days_count= lambda x: x.nunique()
    )
    .rename_axis('address')
    .reset_index()
)

total_stats = (
    total_stats.reset_index()
               .merge(date_stats, on='address', how='left')
               .set_index('address')
)
total_stats['active_days_count'] = total_stats['active_days_count'].fillna(0)

In [ ]:
#7. 거래 상대의 다양성
# 7-1. 보낸 거래 기준으로 고유 toAddress 수
cp_count_sent = (
    node_df
    .groupby('fromAddress')['toAddress']
    .nunique()
    .rename('counterparty_count_sent')
    .reset_index()
    .rename(columns={'fromAddress':'address'})
)

# 7-2. 받은 거래 기준으로 고유 fromAddress 수
cp_count_recv = (
    node_df
    .groupby('toAddress')['fromAddress']
    .nunique()
    .rename('counterparty_count_recv')
    .reset_index()
    .rename(columns={'toAddress':'address'})
)
# 7-3. 기존 total_stats 와 병합
total_stats = (
    total_stats.reset_index()
               .merge(cp_count_sent, on='address', how='left')
               .merge(cp_count_recv, on='address', how='left')
               .set_index('address')
)
total_stats['counterparty_count_sent'] = total_stats['counterparty_count_sent'].fillna(0)
total_stats['counterparty_count_recv'] = total_stats['counterparty_count_recv'].fillna(0)

In [ ]:
total_stats.describe()


,sent_tx_count,sent_tx_amount,recv_tx_count,recv_tx_amount,external_sent_tx_count,external_sent_tx_amount,external_recv_tx_count,external_recv_tx_amount,hour_entropy,active_days_count,counterparty_count_sent,counterparty_count_recv
count,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06,1.600809e+06
mean,5.592510e+00,9.507613e+06,5.592510e+00,9.507613e+06,1.335919e+00,3.561529e+04,1.335919e+00,3.561529e+04,3.046555e-01,4.209709e+00,1.557923e+00,1.557923e+00
std,1.325989e+03,8.836660e+09,6.425674e+02,8.836649e+09,2.406713e+01,2.374436e+07,2.186381e+01,2.374147e+07,7.311899e-01,8.024038e+02,3.294471e+02,1.542524e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,1.000000e+00,4.036174e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,1.000000e+00,4.739720e-01,1.000000e+00,4.490000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,1.000000e+00,5.163077e+01,3.000000e+00,6.296766e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
max,1.448407e+06,1.000000e+13,4.387820e+05,1.000000e+13,1.222000e+04,3.000000e+10,7.350000e+03,3.000000e+10,4.584068e+00,8.104160e+05,3.228280e+05,1.400780e+05


In [ ]:
#8. 거래금액 특성
# 8-1. 평균 송금액
total_stats['sent_tx_amount_mean'] = total_stats['sent_tx_amount'] / total_stats['sent_tx_count']
# 8-2. 평균 수신액
total_stats['recv_tx_amount_mean'] = total_stats['recv_tx_amount'] / total_stats['recv_tx_count']
# 8-3. 평균 외부 송금액
total_stats['external_sent_tx_amount_mean'] = total_stats['external_sent_tx_amount'] / total_stats['external_sent_tx_count']
# 8-4. 평균 외부 수신액
total_stats['external_recv_tx_amount_mean'] = total_stats['external_recv_tx_amount'] / total_stats['external_recv_tx_count']



In [ ]:
total_stats.fillna(0, inplace=True)

/tmp/ipykernel_675883/366128738.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns, Asia/Seoul], please explicitly cast to a compatible dtype first.
  total_stats.fillna(0, inplace=True)


In [ ]:
#to csv
total_stats.to_csv("../data/total_stats_realdata_part_1.csv")

In [ ]:
node_df.to_csv("../data/transfer_data_realdata_part_1.csv")

In [ ]:
node_df.nunique()

type               3
txhash         76032
timestamp      29761
amount         46728
denom            169
dpDenom           80
fromAddress    28196
toAddress      20197
fromChain         26
fromChainId       26
toChain           27
toChainId         27
is_external        2
hour              22
dtype: int64

In [ ]:
node_df.shape

(76229, 14)